In [3]:
from draftbot_sim_adapted import *
import pandas as pd
import numpy as np
import os
from Agents import *

In [4]:
draft_path = 'C:/Users/trist/mtg_research/test_draft.txt'
cards_path = 'C:/Users/trist/mtg_research/mtg_draftbot/data/m20/m20-cards.json'
weights_path = 'C:/Users/trist/mtg_research/mtg_draftbot/data/new_generated_weights.json'

In [5]:
def comp_generator(num_iter,agents,n_cards_in_pack,rounds,cards_path,card_values_path,packs_input_file):
    '''Streamline comparison of drafts and create outputs with player prefs;
    this gives us quick glances into how different arrangements '''

    #Source the number of drafters for the original draft as # items in the agent function list
    n_drafters = len(agents)

    #Create two accumulator lists to hold the preferences 
    ss_prefs = []
    ms_prefs = []

    #Create accumulator to iterate through the number of iterations (num_iter) put into the simulation generator
    r = 1

    #Create loop that will instantiate two draft objects and run them iteratively 
    for r in range(1,num_iter+1):

        #Instantiate the single strategy draft, which we will use as a baseline to compare the MultiStrat draft
        draft_ss = Draft(n_drafters=n_drafters,
        n_rounds=rounds,
        n_cards_in_pack=n_cards_in_pack,
        cards_path=cards_path,
        card_values_path=card_values_path)

        #Instantiate the multi strategy draft, this object will simulate what happens with the exact same
        #card packs, but when the drafters in each sit may or may not use different strategies (e.g. seat 1 uses 
        # strategy A, while the rest use strategy B)
        draft_ms = MultiStratDraft(
        n_rounds=rounds,
        n_cards_in_pack=n_cards_in_pack,
        cards_path=cards_path,
        card_values_path=weights_path,
        packs_input_file=draft_path,
        agent_list=agents)
                    
        #Calling this function will simulate a draft for us. This will allow us to call picks, options, etc. from the draft on a seat by seat basis
        draft_ss.draft()

        #Create an array of zeroes to append values to. We will use this to house the player preferences for a given round. 
        result_array = np.empty(0)

        #This loop will iterate through the drafter preferences and create an output array of shape 8, 12 most commonly. 
        #The 8 here represents the number of drafters (note this is technically n drafters) The other 12 rows 
        #are the n archetypes (most commonly 10), and then 1 column for the player # and one for the draft # (these two remain static)
        for item,idx in enumerate(draft_ss.drafter_preferences):

            #Create a list that includes the drafters preferences across n_archetypes and which draft # this is in the simulator (e.g. draft 5)
            player=np.append(item,r)

            #Now add the player position to the array, this being seat 1,2,3,4 etc. at the drafting table
            #This marks the end of us taking data from a singular drafter
            player=np.append(player,idx)

            #Now, we add the aggregated data to our result array, which holds the prefs of ALL drafters in 1 draft (most commonly this is 8)
            result_array = np.append(result_array,player)
            print(result_array.shape)

        #Append the array to our single strategy list of drafts/draft preferences
        ss_prefs.append(result_array.reshape((draft_ss.n_drafters,draft_ss.n_archetypes)))

        #This logic overwrites any existing pack input file. 
        #Note that we instantiated the draft objects above using this path, but removing/replacing the file here allows us to remove any
        #previous file and ensure that our multi-strat draft runs on the exact same packs
        if os.path.exists(packs_input_file):
                os.remove(packs_input_file)

        #Use helper functio to save the packs array at the filepath coded in the generator
        save_packs_array(draft_ss,packs_input_file)

        #Now, run the multistrat draft using the exact same cards as before
        draft_ms.draft()

        #Create same empty results array for the draft
        result_array = np.empty(0)

        #Iterate through the drafter preferences, storing them all in a numpy array
        for item,idx in enumerate(draft_ms.drafter_preferences):

            #Add prefs + draft number to array
            player=np.append(item,r)

            #Add the drafter seat to the array 
            player=np.append(player,idx)

            #Add one player's results to the array for the overall draft
            result_array = np.append(result_array,player)

        
        ms_prefs.append(result_array.reshape((draft_ms.n_drafters,draft_ms.n_archetypes+2)))
        r += 1


        return (ms_prefs,ss_prefs)

def macro_prefs_sum_stats(draft_comp):
    '''Take a draft comp and output results'''

    ss_prefs = draft_comp[1]
    ms_prefs = draft_comp[0]
    ss_prefs_df = pd.DataFrame(np.concatenate(ss_prefs))
    ms_prefs_df = pd.DataFrame(np.concatenate(ms_prefs))

    ssdf = ss_prefs_df.set_index([0,1])
    ssdf = ssdf.groupby(by=[0]).mean()
    ssdf['maximum'] = ssdf.max(axis=1)
    ssdf['minimum'] = ssdf.min(axis=1)
    ssdf['average'] = ssdf.mean(axis=1)
    ssdf['stdev'] = ssdf.std(axis=1)

    msdf = ms_prefs_df.set_index([0,1])
    msdf = msdf.groupby(by=[0]).mean()
    msdf['maximum'] = msdf.max(axis=1)
    msdf['minimum'] = msdf.min(axis=1)
    msdf['average'] = msdf.mean(axis=1)
    msdf['stdev'] = msdf.std(axis=1)

    return (msdf,ssdf)

In [6]:
agents_onek = [
  Naive_Pass_Agent.decision_function, 
  Basic_Agent.decision_function, 
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

one_k_comp = comp_generator(1000,agents_onek,14,3,cards_path,weights_path,
draft_path)

agents_twosix = [
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function, 
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

one_k_twosix = comp_generator(1000,agents_twosix,14,3,cards_path,weights_path,
draft_path)

agents_threefive = [
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

threefive_k = comp_generator(1000,agents_threefive,14,3,cards_path,weights_path,
draft_path)

agents_fourfour = [
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function,
  Naive_Pass_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]
fourfour_k = comp_generator(1000,agents_fourfour,14,3,cards_path,weights_path,
draft_path)

(12,)
(24,)
(36,)
(48,)
(60,)
(72,)
(84,)
(96,)


ValueError: cannot reshape array of size 96 into shape (8,10)

In [ ]:
nr_agents_onek = [
  Naive_Receiver_Agent.decision_function, 
  Basic_Agent.decision_function, 
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

nr_one_k_comp = comp_generator(1000,nr_agents_onek,14,3,cards_path,weights_path,
draft_path)

nr_agents_twosix = [
  Naive_Receiver_Agent.decision_function, 
  Naive_Receiver_Agent.decision_function, 
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

nr_one_k_twosix = comp_generator(1000,nr_agents_twosix,14,3,cards_path,weights_path,
draft_path)

nr_agents_threefive = [
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

nr_threefive_k = comp_generator(1000,nr_agents_threefive,14,3,cards_path,weights_path,
draft_path)

nr_agents_fourfour = [
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function,
  Naive_Pass_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]
nr_fourfour_k = comp_generator(1000,nr_agents_fourfour,14,3,cards_path,weights_path,
draft_path)

ValueError: cannot reshape array of size 96 into shape (8,10)

In [ ]:
thousand_one = macro_prefs_sum_stats(one_k_comp)
thousand_two = macro_prefs_sum_stats(one_k_twosix)
thousand_three = macro_prefs_sum_stats(threefive_k)
thousand_four = macro_prefs_sum_stats(fourfour_k)

nr_thousand_one = macro_prefs_sum_stats(nr_one_k_comp)
nr_thousand_two = macro_prefs_sum_stats(nr_one_k_twosix)
nr_thousand_three = macro_prefs_sum_stats(nr_threefive_k)
nr_thousand_four = macro_prefs_sum_stats(nr_fourfour_k)

In [ ]:
#Difference in maxes is largest for the position with the new strategy
#Note that a negative reduction here would be analogous to saying that a preference toward the most preferred archetype goes down
#That really means that drafter 0 has a more diverse/less specialized pool because they don't have as strong a favored archetype as the others. 

print(nr_thousand_one[0]['average'] - nr_thousand_one[1]['average'])
print(nr_thousand_two[0]['average'] - nr_thousand_two[1]['average'])

0
0.0   -1.249556
1.0    0.589323
2.0    0.494209
3.0    0.634299
4.0    0.114250
5.0    0.081929
6.0    0.222955
7.0    0.134683
Name: average, dtype: float64
0
0.0   -1.475285
1.0   -1.601932
2.0    1.209593
3.0    0.406771
4.0    0.981686
5.0    0.389978
6.0   -0.080230
7.0    0.269899
Name: average, dtype: float64


In [ ]:
print(thousand_one[0]['average'] - thousand_one[1]['average'])
print(thousand_two[0]['average'] - thousand_two[1]['average'])

0
0.0   -0.464397
1.0    0.155643
2.0   -0.074166
3.0    0.369340
4.0    0.017271
5.0    0.171724
6.0    0.111302
7.0   -0.078026
Name: average, dtype: float64
0
0.0   -0.203315
1.0   -0.791327
2.0    0.192046
3.0    0.046511
4.0    0.032307
5.0    0.108719
6.0    0.066486
7.0   -0.084707
Name: average, dtype: float64


In [ ]:
print(nr_thousand_three[0]['average'] - nr_thousand_three[1]['average'])
print(nr_thousand_four[0]['average'] - nr_thousand_four[1]['average'])

0
0.0   -0.140521
1.0   -0.566671
2.0   -0.721477
3.0    0.599765
4.0    0.400795
5.0   -0.138453
6.0    0.163789
7.0   -0.008663
Name: average, dtype: float64 0
0.0   -0.195954
1.0   -0.476238
2.0   -0.544514
3.0   -0.204719
4.0    0.741415
5.0    0.453924
6.0    0.445974
7.0   -0.038485
Name: average, dtype: float64
0
0.0   -0.195954
1.0   -0.476238
2.0   -0.544514
3.0   -0.204719
4.0    0.741415
5.0    0.453924
6.0    0.445974
7.0   -0.038485
Name: average, dtype: float64


In [ ]:
print(thousand_three[0]['average'] - thousand_three[1]['average'])
print(thousand_four[0]['average'] - thousand_four[1]['average'])

0
0.0   -0.651572
1.0   -0.692191
2.0   -0.414363
3.0    0.746310
4.0    0.395571
5.0   -0.261921
6.0   -0.062020
7.0    0.048860
Name: average, dtype: float64
0
0.0   -0.221952
1.0   -0.339454
2.0   -0.294333
3.0   -0.421811
4.0    1.321017
5.0    0.311100
6.0   -0.065396
7.0    0.441908
Name: average, dtype: float64


In [ ]:
agents_onek = [
  Naive_Pass_Agent.decision_function, 
  Basic_Agent.decision_function, 
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

one_k_comp = comp_generator(1000,agents_onek,14,3,cards_path,weights_path,
draft_path)

agents_twosix = [
  Arch_Pass_Agent.decision_function, 
  Basic_Agent.decision_function, 
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

one_k_twosix = comp_generator(1000,agents_twosix,14,3,cards_path,weights_path,
draft_path)

ValueError: cannot reshape array of size 96 into shape (8,10)